importamos las librerias

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException


In [4]:
import pandas as pd

Indicamos el texto a buscar

In [1]:
texto_a_buscar = 'Tarjeta Grafica'

Bloque de codigo que ejecuta el scraping sobre amazon

In [17]:
# Inicializamos las listas

nombre= []
precios = []
caracteristicas = []
stars = []
ratings = []



# Cargamos el driver

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
url = "https://www.amazon.es/"
driver.get(url)




# Probamos en las diferentes paginas de inicio de amazon

time.sleep(1)

try:
    busqueda = driver.find_element(By.CSS_SELECTOR,'input#twotabsearchtextbox')

    busqueda.clear()
    busqueda.send_keys(texto_a_buscar)
    busqueda.send_keys(Keys.ENTER)

except NoSuchElementException:

    busqueda = driver.find_element(By.CSS_SELECTOR,'input#nav-bb-search')

    busqueda.clear()
    busqueda.send_keys(texto_a_buscar)
    busqueda.send_keys(Keys.ENTER)




# Comenzamos a scrapear la pagina de resultados de amazon

time.sleep(1)
elements = driver.find_elements(By.CSS_SELECTOR,'div.a-section.a-spacing-base')
   
for element in elements:
   
    btn = element.find_element(By.TAG_NAME,"a")

    enlace_url = btn.get_attribute('href')

    driver.execute_script("window.open('" + enlace_url + "', '_blank');")

    # -----------
    # Precio

    price = element.find_element(By.CSS_SELECTOR,"span.a-price").text
    precios.append(price)


    #------------

    # Manejar la nueva ventana
    nuevas_ventanas = driver.window_handles
    driver.switch_to.window(nuevas_ventanas[-1])

    # -----------
    #Titulo

    title = driver.find_element(By.CSS_SELECTOR,"h1#title").text
    nombre.append(title)


    # -----------
    # Caracteristicas

    try:

        charss = driver.find_element(By.CSS_SELECTOR, 'table#productDetails_techSpec_section_1')


        # Crear un diccionario para almacenar las etiquetas y valores
        info_dict = {}



        # Iterar sobre las filas de la tabla
        for fila in charss.find_elements(By.TAG_NAME, 'tr'):

            try:
            # Obtener la etiqueta y el valor
            
                etiqueta = fila.find_element(By.CLASS_NAME, 'a-size-base.prodDetSectionEntry').text.strip()
                valor = fila.find_element(By.CLASS_NAME, 'a-size-base.prodDetAttrValue').text.strip()

                # Agregar la información al diccionario
                info_dict[etiqueta] = valor

            except NoSuchElementException:
                continue


        caracteristicas.append(info_dict)

    except NoSuchElementException:

        caracteristicas.append(None)



    #----------------
    # Stars and rating

    try:
        stars_and_ratings = driver.find_element(By.CSS_SELECTOR, 'div#averageCustomerReviews').text.split('\n')

        starsss = stars_and_ratings[0]
        ratingsss = stars_and_ratings[-1]

        stars.append(starsss)

        ratings.append(ratingsss)

    except NoSuchElementException:

        stars.append(None)

        ratings.append(None)


    # Cerramos la ventana
    driver.close()
    driver.switch_to.window(nuevas_ventanas[0])


    

WebDriverException: Message: unknown error: cannot determine loading status
from target frame detached
  (Session info: chrome=119.0.6045.123)
Stacktrace:
#0 0x555ea2b5b5e3 <unknown>
#1 0x555ea281df0d <unknown>
#2 0x555ea2803f4e <unknown>
#3 0x555ea2802481 <unknown>
#4 0x555ea2802e39 <unknown>
#5 0x555ea28141d9 <unknown>
#6 0x555ea282b4c1 <unknown>
#7 0x555ea283113b <unknown>
#8 0x555ea28035ce <unknown>
#9 0x555ea282b157 <unknown>
#10 0x555ea28aea48 <unknown>
#11 0x555ea2892cc3 <unknown>
#12 0x555ea285e0e4 <unknown>
#13 0x555ea285f0ae <unknown>
#14 0x555ea2b21ce1 <unknown>
#15 0x555ea2b25b7e <unknown>
#16 0x555ea2b0f4b5 <unknown>
#17 0x555ea2b267d6 <unknown>
#18 0x555ea2af2dbf <unknown>
#19 0x555ea2b49748 <unknown>
#20 0x555ea2b49917 <unknown>
#21 0x555ea2b5a773 <unknown>
#22 0x7f0049e8f6ba <unknown>


Limpiamos las listas antes del dataframe

In [18]:
print(len(nombre))
print(len(precios ))
print(len(caracteristicas ))
print(len(stars ))
print(len(ratings ))

2
3
2
2
2


In [19]:
precios.pop()

'116\n00€'

In [20]:
amazon = {
    'name':nombre,
    'price':precios,
    'characteristics':caracteristicas,
    'stars':stars,
    'ratings':ratings,
}

df = pd.DataFrame(amazon)
df

,name,price,characteristics,stars,ratings
0,GTX1050TI 4GB tarjeta gráfica DDR5 128BIT dobl...,115\n00€,"{'Marca': 'Generic', 'Fabricante': 'jinli', 'C...",None,None
1,PELADN AMD Radeon RX580 8GB Tarjeta Gráfica (G...,119\n00€,"{'Marca': 'Genérico', 'Fabricante': 'Peladn', ...","3,0",4 valoraciones
